In [17]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

In [33]:
# Load your Excel file
file_path = "WA_Postcodes_with_latlong.xlsx"
df = pd.read_excel(file_path, sheet_name="wa")

In [35]:
# Initialize geocoder
geolocator = Nominatim(user_agent="postcode_checker")

In [37]:
# Function to geocode WA postcodes with retry logic
def get_coordinates(locality, postcode):
    query = f"{locality}, WA {postcode}, Australia"
    try:
        location = geolocator.geocode(query, timeout=10)
        if location:
            return location.latitude, location.longitude
    except GeocoderTimedOut:
        time.sleep(1)
        return get_coordinates(locality, postcode)
    return None, None

# Apply geocoding for WA
lat_list = []
long_list = []

for idx, row in df.iterrows():
    lat, long = get_coordinates(row["locality"], row["postcode"])
    lat_list.append(lat)
    long_list.append(long)
    print(f"Processed {idx + 1}/{len(df)}: {row['locality']} {row['postcode']} WA")

Processed 1/1940: GIBSON DESERT NORTH 872 WA
Processed 2/1940: GIBSON DESERT SOUTH 872 WA
Processed 3/1940: IRRUNYTJU 872 WA
Processed 4/1940: KANPA 872 WA
Processed 5/1940: KIWIRRKURRA 872 WA
Processed 6/1940: MANTAMARU 872 WA
Processed 7/1940: NGAANYATJARRA-GILES 872 WA
Processed 8/1940: PAPULANKUTJA 872 WA
Processed 9/1940: PATJARR 872 WA
Processed 10/1940: TJIRRKARLI 872 WA
Processed 11/1940: TJUKURLA 872 WA
Processed 12/1940: WARAKURNA 872 WA
Processed 13/1940: WINGELLINA 872 WA
Processed 14/1940: CITY DELIVERY CENTRE 6000 WA
Processed 15/1940: PERTH 6000 WA
Processed 16/1940: PERTH GPO 6000 WA
Processed 17/1940: PERTH 6001 WA
Processed 18/1940: HIGHGATE 6003 WA
Processed 19/1940: NORTHBRIDGE 6003 WA
Processed 20/1940: EAST PERTH 6004 WA
Processed 21/1940: KINGS PARK 6005 WA
Processed 22/1940: WEST PERTH 6005 WA
Processed 23/1940: NORTH PERTH 6006 WA
Processed 24/1940: LEEDERVILLE 6007 WA
Processed 25/1940: WEST LEEDERVILLE 6007 WA
Processed 26/1940: DAGLISH 6008 WA
Processed 27/1

In [39]:
# Add geocoded values to DataFrame
df["geo_lat"] = lat_list
df["geo_long"] = long_list

In [41]:
# Compute difference
df["lat_diff"] = (df["lat"] - df["geo_lat"]).abs()
df["long_diff"] = (df["long"] - df["geo_long"]).abs()

In [43]:
# Flag suspect records (e.g., more than 0.01 degree difference ≈ ~1 km)
df["flag"] = (df["lat_diff"] > 0.01) | (df["long_diff"] > 0.01)# Flag suspect records (e.g., more than 0.01 degree difference ≈ ~1 km)
df["flag"] = (df["lat_diff"] > 0.01) | (df["long_diff"] > 0.01)

In [45]:
# Save results
df.to_excel("postcode_verification_results_wa.xlsx", index=False)
print("Verification complete. Results saved to 'postcode_verification_results_wa.xlsx'.")

Verification complete. Results saved to 'postcode_verification_results_wa.xlsx'.
